In [1]:
import os 
import pandas as pd 
import numpy as np
import json
import re

In [2]:
par_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(par_dir, '2.2 NER - analysis by entity type')

stanza_dir = os.path.join(data_dir, "df_entities_spacy_processed.csv")
spacy_dir = os.path.join(data_dir, "df_entities_stanza_processed.csv")

entities_stanza = pd.read_csv(stanza_dir)
entities_spacy = pd.read_csv(spacy_dir)

In [3]:
list_people = list(set(list(np.unique(entities_spacy["subject"])) + list(np.unique(entities_stanza["subject"]))))
list_entities = list(set(list(np.unique(entities_spacy["label"])) + list(np.unique(entities_stanza["label"]))))

In [4]:
def make_regex(entity):
    entity_extra = entity.split(" ")
    entity_new = "_?".join(entity_extra) # add optional _ since the entities usually appear that way in the graph
    
    return entity_new

In [50]:
for person in list_people[0:3]:
    
    # filtering each df by the subject of the bio
    filter_df_spacy = entities_spacy[entities_spacy["subject"] == person]
    filter_df_stanza = entities_stanza[entities_stanza["subject"] == person]
    
    # getting the category to know which Data Directory subfolder we have to access
    category = filter_df_spacy["category"].iloc[0]
    
    # print(category)
    # getting the entities found in the bios per package
    set_entities_spacy = set(filter_df_spacy["text"].to_list())
    set_entities_stanza = set(filter_df_stanza["text"].to_list())
    
    # total amount of unique entities
    total_entities_spacy = len(set_entities_spacy)
    total_entities_stanza = len(set_entities_stanza)
    
    found_entities_spacy = 0
    found_entities_stanza = 0   
    
    # loading the json graphs to query them
    par_dir = os.path.dirname(os.path.dirname(os.getcwd()))
    name_file_graph = person.replace(" ", "__") + ".json"
    data_dir = os.path.join(par_dir, "Data Directory")
    
    if category == "Chemistry":
        graph_dir = os.path.join(data_dir, "chemistry_nobel_laureate")
    
    else:
        graph_dir = os.path.join(data_dir, "physics_nobel_laureate")
        
    person_graph_path = os.path.join(graph_dir, name_file_graph)
    # print(person_graph_path)
    
    with open(person_graph_path, "r") as f:
        personal_graph = json.load(f)
    # if f:
    #     print("Graph load successfully!")
    
    list_subjects = [dict_graph["subject"]["value"].strip("http://dbpedia.org/resource/") for dict_graph in personal_graph]
    list_objects = [dict_graph["object"]["value"].strip("http://dbpedia.org/resource/") for dict_graph in personal_graph]
    
    list_entities_graph = list(set(list_subjects + list_objects))
    
    # print(list_entities_graph)
    
    regex_entities_stanza = [ent for ent in set_entities_stanza if ent.count("(") == ent.count(")")]
    print(regex_entities_spacy)
    regex_entities_spacy = [ent for ent in set_entities_spacy if ent.count("(") == ent.count(")")]
    
    matches_spacy = 0
    
    print(regex_entities_stanza)
    for ent in list_entities_graph:
        
        match_search = [re.search(regex, ent, flags=re.IGNORECASE) for regex in regex_entities_spacy]
        match = [(search.group(0), search) for search in match_search if search != None]
        print(ent)
        print(match)
        if match:
            matches_spacy = matches_spacy + 1
        
        print(matches_spacy)
    prop_matches = round((matches_spacy/total_entities_spacy), 2)

['Victor Meyer', 'Wolcott Gibbs Memorial Laboratory', 'International Atomic Weights Committee', 'Theodore William Richards', '1883', 'Harvard University Press.', 'Miriam Stuart Thayer', '1911', '1913', 'National Academy of Sciences', 'Bachelor of Science degree', '1915', 'Harvard University', 'the United States', 'Davy Medal', '1910', '1885', '1917', 'the American Association for the Advancement of Science', 'Cooke', 'two years', 'Bachelor of Arts degree', 'the University of Göttingen', 'Theodore Richards Medal', 'Jakob Berzelius\nFarrington Daniels', 'Josiah Parsons Cooke', 'Massachusetts', 'Le Blanc Medal', 'James Bryant', 'Journal of the American Chemical Society. 37', '1901', '1643', '286', 'Cambridge', 'Member of the American Philosophical Society', 'Washington', 'Newport', '1974', 'Royal Irish Academy', 'Nobelprize.org', '1923', '1656', 'Carnagie Institution', 'William Theodore', '1916', 'John Henry', 'Forbes', '15,000', '1912', 'Richards', '1896', 'Theodore Richards', 'Greenough